In [1]:
import numpy as np
import pandas as pd

In [59]:
FAANG_tickers = ['FB', 'AAPL', 'AMZN', 'NFLX', 'GOOG']
tech_etf_ticker = ['VGT', 'QQQ', 'TQQQ', 'FNGU']
my_tickers = ['AAPL', 'AMZN', 'BABA', 'BIGC', 'DOCU', 'ETSY', 'FB', 'GOOG', 'JD',
              'MSFT', 'NFLX', 'NIO', 'NVDA', 'PINS', 'PYPL', 'ROKU', 'SHOP', 'SPLK', 'SQ', 'UBER', 'LYFT', 'W', 'WMT', 'ZM']

print("Number of Stock {}. Number of ETFs {}".format(
    len(my_tickers), len(tech_etf_ticker)))

tickers = my_tickers + tech_etf_ticker
print(tickers)

Number of Stock 24. Number of ETFs 4
['AAPL', 'AMZN', 'BABA', 'BIGC', 'DOCU', 'ETSY', 'FB', 'GOOG', 'JD', 'MSFT', 'NFLX', 'NIO', 'NVDA', 'PINS', 'PYPL', 'ROKU', 'SHOP', 'SPLK', 'SQ', 'UBER', 'LYFT', 'W', 'WMT', 'ZM', 'VGT', 'QQQ', 'TQQQ', 'FNGU']


In [13]:
from datetime import date
from datetime import timedelta

today = '{}'.format(date.today())
y1_ago = '{}'.format(date.today() - timedelta(days = 365))

print([today, y1_ago])

['2020-09-05', '2019-09-06']


In [88]:
import yfinance as yf
tickers_str = ' '.join(tickers)

data = yf.download(tickers_str, y1_ago, today, group_by='ticker')

type(data)

[*********************100%***********************]  28 of 28 completed


pandas.core.frame.DataFrame

In [107]:
print("Closing Prices")
c_prices = data.iloc[-1].loc[(slice(None), 'Close')].sort_index()
print(c_prices.round(2))

Closing Prices
AAPL    120.96
AMZN   3294.62
BABA    281.39
BIGC    106.98
DOCU    216.26
ETSY    112.04
FB      282.73
FNGU    194.50
GOOG   1591.04
JD       80.00
LYFT     28.97
MSFT    214.25
NFLX    516.05
NIO      17.98
NVDA    504.90
PINS     34.38
PYPL    191.84
QQQ     283.58
ROKU    159.91
SHOP    973.02
SPLK    200.14
SQ      146.39
TQQQ    141.64
UBER     33.24
VGT     313.29
W       260.94
WMT     142.83
ZM      369.89
Name: 2020-09-04 00:00:00, dtype: float64


In [102]:
y1_ago_price = data.iloc[0].loc[(slice(None), 'Close')]
m6_ago_price = data.iloc[-26 * 5].loc[(slice(None), 'Close')]
m3_ago_price = data.iloc[-13 * 5].loc[(slice(None), 'Close')]
m1_ago_price = data.iloc[-4 * 5].loc[(slice(None), 'Close')]
curr_price = data.iloc[-1].loc[(slice(None), 'Close')]

In [103]:
m1_gain = 100 * (curr_price - m1_ago_price) / m1_ago_price
m3_gain = 100 * (curr_price - m3_ago_price) / m3_ago_price
m6_gain = 100 * (curr_price - m6_ago_price) / m6_ago_price
y1_gain = 100 * (curr_price - y1_ago_price) / y1_ago_price

In [104]:
curr_date = np.datetime_as_string(data.index.values[-1], unit='D')

In [126]:
#pd.options.display.float_format = '{:.2f}'.format
df = pd.DataFrame([curr_price, m1_gain, m3_gain, m6_gain, y1_gain],
                  index=[curr_date,  '1m', '3m', '6m', '1y']).round(2)
df.T.sort_values(by='1m', ascending=False)

,2020-09-04,1m,3m,6m,1y
ZM,369.89,47.96,78.17,216.69,333.08
BIGC,106.98,39.44,nan,nan,nan
FNGU,194.50,37.26,127.49,141.40,427.96
JD,80.00,30.10,35.50,82.19,157.40
NIO,17.98,26.53,221.65,364.60,503.36
BABA,281.39,13.40,28.17,32.76,59.26
TQQQ,141.64,13.06,57.57,59.34,119.94
NVDA,504.90,13.05,41.51,77.46,182.62
DOCU,216.26,9.01,54.87,141.50,284.33
WMT,142.83,8.30,17.50,22.32,24.49


In [109]:
t = 'FNGU'
dates = np.datetime_as_string(data.index.values, unit='D')
print(dates[0], y1_ago_price[t])
print(dates[-26 * 5], m6_ago_price[t])
print(dates[-13 * 5], m3_ago_price[t])
print(dates[-1], curr_price[t])

2019-09-06 36.84000015258789
2020-03-04 80.56999969482422
2020-06-05 85.5
2020-09-04 194.5


In [157]:
t1 = '2020-09-02' 
t2 = '2020-09-04'

price1 = data.loc[t1].loc[(slice(None), 'Close')]
price2 = data.loc[t2].loc[(slice(None), 'Close')]
price1.name = t1
price2.name = t2

diff = (100 * (price2 - price1) / price1)
diff.name = "Gain"

In [158]:
change = pd.concat([price1, price2, diff], axis=1)
change.sort_values(by='Gain', ascending=False)

,2020-09-02,2020-09-04,Gain
LYFT,28.61,28.97,1.26
WMT,147.68,142.83,-3.28
UBER,34.37,33.24,-3.29
JD,83.21,80.00,-3.86
BIGC,112.00,106.98,-4.48
BABA,296.07,281.39,-4.96
QQQ,302.76,283.58,-6.34
FB,302.50,282.73,-6.54
NFLX,552.84,516.05,-6.65
AMZN,3531.45,3294.62,-6.71
